In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# Parameters
time_steps = 5          # Number of frames in sequence (e.g. 5 images per sample)
img_height = 64
img_width = 64
channels = 3
num_classes = 10

# Dummy data: 1000 samples, each sample is a sequence of 5 images (64x64x3)
x_data = np.random.random((1000, time_steps, img_height, img_width, channels))
y_data = np.random.randint(0, num_classes, size=(1000,))

# Build CRNN model
input_layer = layers.Input(shape=(time_steps, img_height, img_width, channels))

# TimeDistributed CNN layers to process each frame independently but share weights
cnn = layers.TimeDistributed(layers.Conv2D(32, (3,3), activation='relu'))(input_layer)
cnn = layers.TimeDistributed(layers.MaxPooling2D((2, 2)))(cnn)
cnn = layers.TimeDistributed(layers.Conv2D(64, (3,3), activation='relu'))(cnn)
cnn = layers.TimeDistributed(layers.MaxPooling2D((2, 2)))(cnn)
cnn = layers.TimeDistributed(layers.Flatten())(cnn)

# Now, cnn output shape: (batch_size, time_steps, features)
# Pass sequence of features to RNN
rnn = layers.SimpleRNN(64, activation='tanh')(cnn)

# Output layer for classification
output = layers.Dense(num_classes, activation='softmax')(rnn)

# Define and compile model
model = models.Model(inputs=input_layer, outputs=output)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
model.fit(x_data, y_data, epochs=5, batch_size=32, validation_split=0.2)

# Evaluate
loss, accuracy = model.evaluate(x_data, y_data)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")


Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.0987 - loss: 2.7116 - val_accuracy: 0.1000 - val_loss: 2.3199
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.1116 - loss: 2.3284 - val_accuracy: 0.1000 - val_loss: 2.3007
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.1199 - loss: 2.3121 - val_accuracy: 0.1050 - val_loss: 2.3370
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.0815 - loss: 2.3125 - val_accuracy: 0.1050 - val_loss: 2.3089
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.1011 - loss: 2.3074 - val_accuracy: 0.1050 - val_loss: 2.3135
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 285ms/step - accuracy: 0.1176 - loss: 2.2976
Loss: 2.3021, Accuracy: 0.1120
